In [38]:
import numpy as np
import pandas as pd


class gm11(object):
    """
    经典GM(1,1)模型
    
    使用方法：
    1.实例化类     model = gm11(data,predstep=2) 需要修改predstep
    2.训练模型     model.fit()
    3.查看拟合误差  model.MSE()
    4.预测        model.predict()

    Ps:背景值系数bg_coff接收的是一个列表（主要为了后面新模型的构建），默认值为一个空列表，此时背景值系数默认全部为0.5

    输入数据：'GM11示例数据.xlsx'：原始数据  n行1列数据
    
    """
    def __init__(self, sys_data: pd.DataFrame, predstep: int = 2, bg_coff: list = []):
        
        #检测输入的sys_data是否为Dataframe类型
        if not isinstance(sys_data, pd.DataFrame):
            raise ImportError('请使用Dataframe格式载入数据')
        #检测输入的predstep是否为整数类型
        if not isinstance(predstep,int):
            raise ValueError('predstep必须是整数')
            
        # if bg_coff > 1 or bg_coff < 0:
        #     raise ValueError('background_coff必须在[0,1]之间')

        self.data = np.array(sys_data.iloc[:, 0].values)     # sys_data:(行数*列数)---->self.data:一维数组，（行数，）      
        self.data_shape = self.data.shape[0]                 #self.data_shape：self.data的行数 
        self.data = self.data.reshape((self.data_shape,1))   #self.data：一维数组，（行数，）---->(行数*1)
        
        self.coff = []
        self.sim_values = np.zeros((self.data_shape,1))
        self.predstep = predstep
        self.pred_values = np.zeros((self.predstep,1))
        self.error = []
        self.bg_coff = bg_coff
        self.rel_errors = []

    #数据检验（是否适合灰色预测）
    def data_check(self):
        """
        parameter:
        x1:累加序列
        sigma：累加序列的级比
        功能：验证累加序列的级比是否符合准指数规律
        """
        # 1. 累加生成序列 x^{(1)}
        x1 = np.cumsum(self.data)
    
        # 2. 计算累加序列的级比 σ(k) = x1[k] / x1[k-1]
        sigma = []
        for i in range(1, self.data_shape):
            sigma.append(x1[i] / x1[i-1])
    
        # 3. 计算级比区间阈值（理论区间应为 (1, 1.5)）
        valid_min, valid_max = 1.0, 1.5
    
        # 4. 检验级比是否在区间内
        valid_count = 0
        for s in sigma:
            if valid_min <= s <= valid_max:
                valid_count += 1
    
        # 5. 判断通过率（建议后 90% 数据满足条件）
        pass_rate = valid_count / len(sigma)
        if pass_rate >= 0.9:
            print("该数据可以用灰色 GM(1,1) 模型（级比通过率：{:.2%}）".format(pass_rate))
        else:
            print("该数据不适用灰色 GM(1,1) 模型（级比通过率：{:.2%}）".format(pass_rate))

    #最小二乘法求参数a，b
    def least_squares_method_function(self):
        '''
        方法解释：
        求解𝑥0(t)=−𝑎*z1(t)+b中的a，b
        将方程化为Y=X*β+误差项，其中X和β为矩阵
        
        parameter：
        data_leijia：（对原始数据的）累加数据集
        Y：从第二个元素开始的原始数据，对应方程中的Y
        self.coff:(2,1)系数矩阵
        B:对应方程中的X
       
        '''
        #对原始数据累加
        data_leijia = np.cumsum(self.data)  
        Y = self.data[1:]
        
        # 计算背景值
        background_values = np.zeros((self.data_shape - 1, 1)).reshape((self.data_shape - 1, 1))  #background_values：零矩阵，(行数-1，1)
        if self.bg_coff == []:
            for i in range(1,self.data_shape):
                background_values[i-1][0] = 0.5 * data_leijia[i] + (1-0.5) * data_leijia[i-1]
        else:
            for i in range(1,self.data_shape):
                background_values[i-1][0] = (1-self.bg_coff[i-1]) * data_leijia[i] + self.bg_coff[i-1] * data_leijia[i-1]
                
        one_array = np.ones((self.data_shape-1,1)).reshape((self.data_shape-1,1))
        background_values = -background_values
        B = np.hstack((background_values,one_array))
        
        # 用最小二乘求解参数,最小二乘参数估计公式：coff = (B^T*B)^(-1)*(B^T*Y)
        self.coff = np.matmul(np.linalg.inv(np.matmul(B.T, B)), np.matmul(B.T, Y))

    #模型拟合
    def fit(self) -> np.array:
        '''
        parameter:
        self.sim_values:预测数据

        '''
        self.least_squares_method_function()
        a = self.coff[0][0]
        b = self.coff[1][0]
        # 求解拟合值
        self.sim_values[0] = self.data[0][0]
        for i in range(2,self.data_shape+1):
            #x0(m+1)=(1-e^a)*[x0(1)-b/a]*e^(-a*m)
            self.sim_values[i-1] = (1 - np.exp(a)) * (self.data[0][0] - b / a) * np.exp(-a * (i - 1))
        self.sim_values = self.sim_values.reshape((1,self.data_shape))[0]
        return self.sim_values

    #模型预测
    '''
    parameter:
    self.pred_values:预测值

    '''
    def predict(self) -> np.array:
        self.least_squares_method_function()
        a = self.coff[0][0]
        b = self.coff[1][0]
        # 求解预测值
        for i in range(self.data_shape+1,self.data_shape+1+self.predstep):
            self.pred_values[i - 1 - self.data_shape][0] = (1 - np.exp(a)) * (self.data[0][0] - b / a) * np.exp(-a * (i - 1))
        self.pred_values = self.pred_values.reshape((1,self.predstep))[0]
        return self.pred_values

    #计算相对残差和平均相对残差
    def loss(self) -> list:
        '''
    parameter:
    self.error:相对误差
    error_pingjun:平均相对残差
    
    输出：平均相对残差error_pingjun
    '''
        for i in range(self.data_shape):
            self.error.append(abs(self.sim_values[i]-self.data[i][0])/self.data[i][0])
        error_pingjun=sum(self.error)/len(self.error)
        return error_pingjun

    #计算绝对残差
    '''
    parameters:
    self.rel_errors:绝对残差

    输出：绝对残差self.rel_errors
    '''
    def errors_juedui(self):
        for i in range(self.data_shape):
            self.rel_errors.append(self.data[i][0]-self.sim_values[i])
        return self.rel_errors

    #模型评价
    def model_pingjia(self):
        error_pingjun=self.loss()
        if error_pingjun<0.1:
             print('对原数据的拟合效果非常不错')
        elif error_pingjun<=0.2 and error_pingjun>0.1:
            print('对原数据的拟合达到一般要求')
        elif error_pingjun>0.2:
            print('对原数据的拟合不太理想，需要调整')

if __name__ == '__main__':
    '''
    输入：'GM11示例数据.xlsx'：原始数据  n行1列数据
    '''
    data = pd.read_excel('GM11示例数据.xlsx',header=None)
    model= gm11(data)
    print('GM(1,1)的拟合值是： ', model.fit())
    print(f'GM(1,1)的{model.predstep}步预测值是： ', model.predict())
    print('GM(1,1)的平均相对残差是：', model.loss())
    model.model_pingjia()
    print('绝对误差为：',model.errors_juedui())
    model.data_check()

GM(1,1)的拟合值是：  [120.         132.95843051 140.97745716 149.48012962 158.49561768
 168.05485042 178.19062232 188.93770578 200.33297038 212.41550943]
GM(1,1)的2步预测值是：  [225.226774  238.8107152]
GM(1,1)的平均相对残差是： 0.008285964120396724
对原数据的拟合效果非常不错
绝对误差为： [0.0, -2.9584305140828633, -0.9774571617436436, 0.5198703764319816, 1.504382316649128, 1.9451495788226794, 1.8093776793993754, 1.0622942246137654, -0.33297038179443916, -2.4155094256536813]
该数据不适用灰色 GM(1,1) 模型（级比通过率：77.78%）
